In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [4]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


4301

### Dividindo Documentos - Splitting/Chunking

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [6]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [10]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [12]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [14]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [17]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como um dos benefícios para os colaboradores. Além disso, há também plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [27]:
import requests
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

url = 'https://strateegia.digital'
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
data = [p.get_text() for p in soup.find_all('p')]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents([Document(page_content=str(data))])


vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

system_template = """Você é um bot de ajuda sobre a plataforma strateegia. Responda as perguntas do usuário de maneira amigável e se mantendo apenas sobre a plataforma..

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)
llm = ChatOpenAI(model="gpt-4o-mini")
rag_chain = (
     {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)
for chunk in rag_chain.stream("Quais são as principais funcionalidades do Strateegia?"):
    print(chunk, end="", flush=True)


O Strateegia possui diversas funcionalidades que visam otimizar a tomada de decisão e a inovação em comunidades colaborativas. Aqui estão algumas das principais:

1. **Debates com Agentes Inteligentes**: A plataforma permite que pessoas debatam com agentes inteligentes (IA), facilitando discussões enriquecedoras e trazendo novas perspectivas.

2. **Assistentes Inteligentes**: Utilize assistentes executivos, mentores e especialistas de diferentes áreas para obter insights valiosos e aplicar esses conhecimentos em seus projetos.

3. **Modo Anônimo Temporário**: Ative essa funcionalidade para que todos os participantes se sintam à vontade para expressar opiniões e discordar, promovendo uma cultura de inovação.

4. **Análises Multidimensionais**: Os assistentes de IA analisam os debates, resumem conteúdos e destacam propostas-chave, ajudando a potencializar suas decisões com análises precisas.

5. **Documentos Prontos para Publicação**: Transforme os resultados dos debates em documentos fo